In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv("../data/processed/ethiopia_fi_enriched.csv")

# Split record types
events = data[data["record_type"] == "event"].copy()
impact_links = data[data["record_type"] == "impact_link"].copy()
observations = data[data["record_type"] == "observation"].copy()


In [2]:
impact_links.head()


,record_id,record_type,category,pillar,indicator,indicator_code,indicator_direction,value_numeric,value_text,value_type,...,impact_direction,impact_magnitude,impact_estimate,lag_months,evidence_basis,comparable_country,collected_by,collection_date,original_text,notes
45,NaN,impact_link,NaN,usage,4G network expansion,NaN,NaN,NaN,NaN,NaN,...,positive,medium,NaN,12.0,Improved connectivity increases digital paymen...,NaN,NaN,NaN,NaN,NaN


In [ ]:
Impact links define how specific events influence financial inclusion indicators.
Each record specifies the affected indicator, impact direction, magnitude, and lag.

In [3]:
impact_events = impact_links.merge(
    events[["indicator", "category", "observation_date"]],
    on="indicator",
    how="left",
    suffixes=("", "_event")
)

impact_events.rename(columns={
    "indicator": "event_name",
    "observation_date": "event_date"
}, inplace=True)

impact_events.head()


,record_id,record_type,category,pillar,event_name,indicator_code,indicator_direction,value_numeric,value_text,value_type,...,impact_estimate,lag_months,evidence_basis,comparable_country,collected_by,collection_date,original_text,notes,category_event,observation_date_event
0,NaN,impact_link,NaN,usage,4G network expansion,NaN,NaN,NaN,NaN,NaN,...,NaN,12.0,Improved connectivity increases digital paymen...,NaN,NaN,NaN,NaN,NaN,infrastructure,2022-06-01


In [4]:
impact_summary = impact_events[[
    "event_name",
    "category",
    "related_indicator",
    "impact_direction",
    "impact_magnitude",
    "lag_months"
]]

impact_summary


,event_name,category,related_indicator,impact_direction,impact_magnitude,lag_months
0,4G network expansion,NaN,USG_DIGITAL_PAYMENT,positive,medium,12.0


Each event is modeled as an exogenous shock with:

a direction (positive/negative)

a magnitude (small/medium/large)

a time lag

Effects are assumed to accumulate gradually rather than instantaneously.